In [11]:
using Pkg
Pkg.add("FileIO")
Pkg.add("MAT")
Pkg.add("Plots")
Pkg.add("Glob")
Pkg.add("Flux")
Pkg.add("MLDataUtils")
Pkg.add("LIBSVM")
Pkg.add("ScikitLearn")
Pkg.add("DataFrames")
using FileIO, MAT, Plots, Glob, Statistics, MLDataUtils, LIBSVM, DataFrames
using ScikitLearn
using ScikitLearn.CrossValidation: train_test_split
using ScikitLearn.Utils: meshgrid

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Chan

In [12]:
# Define a function to read a single MATLAB file
function read_mat_file(filename)
    mat = matread(filename)
    # do something with the contents of the MATLAB file
    return mat
end

read_mat_file (generic function with 1 method)

In [13]:
function read_mat_files(filenames)
    data = []
    for filename in filenames
        push!(data, read_mat_file(filename))
    end
    return data
end

read_mat_files (generic function with 1 method)

In [14]:
filenames = readdir("/Users/byronpetselis/Desktop/Cancer_Data/ben", join=true, sort=false)
# read all the files in this directory

1052-element Vector{String}:
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_I390_lesion_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_I390_normal_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_I452_lesion_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_I452_normal_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_I500_lesion_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_I500_normal_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_afast390_lesion_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_afast390_normal_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_afast452_lesion_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_afast452_normal_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_afast500_lesion_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/Q10_afast500_normal_masked.mat"
 "/Users/byronpetselis/Desktop/Cancer_Data/ben/

In [24]:
#Benign features
Benign_data_labels = ["Q1","Q2","Q4","Q5","Q6","Q7","Q9","Q10","Q11","Q13","Q14","Q16","Q20","Q21", "Q27","Q34","Q35","Q37","Q38","Q40","Q42","Q43","Q46","Q49","Q50", "Q52","Q54","Q56","Q59","Q62","Q63","Q65","Q66","Q67","Q69"]
# lesion_feature_labels = 

Benign_dict = Dict()
Normal_dict = Dict()
for i in Benign_data_labels
    data_lesion = read_mat_files(glob("$i"*"_afast390_lesion*","/Users/byronpetselis/Desktop/Cancer_Data/ben"))
    data_normal = read_mat_files(glob("$i"*"_afast390_normal*","/Users/byronpetselis/Desktop/Cancer_Data/ben"))
    push!(Benign_dict,"$i"=>values(data_lesion[1]["afast390m"]))
    push!(Normal_dict,"$i"=>values(data_normal[1]["afast390nm"]))
end

println(typeof(Benign_dict))
println(typeof(Benign_dict["Q1"]))
println(typeof(Benign_dict["Q1"][1]))

data_lesion = Dict()
data_normal = Dict()

for i in Benign_data_labels
    push!(data_lesion, "$i" => collect(values(Benign_dict["$i"])))
    push!(data_normal, "$i" => collect(values(Normal_dict["$i"])))
end

Dict{Any, Any}
Matrix{Float64}
Float64


In [25]:
df_normal = DataFrame(data_normal)
df_lesion = DataFrame(data_lesion)
# Training testing seperation (Testing 1 Feature: I390)

println(df_normal)

normal_test = []
normal_train = []

benign_test = []
benign_train = []

for key in ["Q10", "Q20", "Q35", "Q43", "Q50", "Q56", "Q69"]
    data = pop!(data_normal, key)
    push!(normal_test, data')
end

for (key,value) in data_normal
    data = pop!(data_normal, key)
    push!(normal_train, data')
end

for key in ["Q10", "Q20", "Q35", "Q43", "Q50", "Q56", "Q69"]
    data = pop!(data_lesion, key)
    push!(benign_test, data')
end

for (key,value) in data_lesion
    data = pop!(data_lesion, key)
    push!(benign_train, data')
end

# normal_train, normal_test = splitobs(df_normal, at = 0.8)
# benign_train, benign_test = splitobs(shuffleobs(df_lesion), at = 0.8)

LoadError: ArgumentError: adding AbstractArray other than AbstractVector as a column of a data frame is not allowed

In [7]:
normal_test

LoadError: UndefVarError: normal_test not defined

In [8]:
X    = hcat(normal_train, benign_train)
println(sizeof(normal_train))
println(sizeof(benign_train))
println(typeof(normal_train))
println(typeof(normal_train))
println(normal_test)
# Y    = vcat(ones(size(benign_train,1)),zeros(size(normal_train,1)))

LoadError: UndefVarError: normal_train not defined

In [9]:
svm_model = svmtrain(X, Y, kernel_type=RBF(gamma=0.1))

LoadError: UndefVarError: RBF not defined